In [1]:
from bsp_file import bspfile, test as bsp_test
bsp_test()

All tests: PASSED


In [11]:
with open("/home/rantrave/games/Quake3/defrag/maps/pama.bsp", "rb") as f:
    map = f.read()

In [27]:
with open("/home/rantrave/games/Quake3/baseq3/maps/detailtest2.bsp", "wb") as f:
    f.write(map)

In [12]:
bsp = bspfile()
bsp.read(map)

0x49425350 46 
560844, 556  
208, 360  
568, 17120  
134040, 87120  
17688, 116352  
240984, 24768  
265752, 3964  
269716, 120  
221160, 3816  
224976, 16008  
269836, 205832  
561400, 17460  
561400, 0  
475668, 85176  
560844, 0  
0, 0  
560844, 0 


In [10]:
list(enumerate(bsp.textures))

[(0, texture: 'textures/NULL' [524288] [1]),
 (1, texture: 'noshader' [0] [1]),
 (2, texture: 'noshader' [524288] [1]),
 (3, texture: 'textures/common/caulk' [527520] [1]),
 (4, texture: 'textures/common/trigger' [524416] [1])]

In [13]:
bsp.textures

[texture: 'textures/NULL' [524288] [1],
 texture: 'noshader' [0] [1],
 texture: 'noshader' [524288] [1],
 texture: 'textures/common/caulk' [527520] [1],
 texture: 'textures/common/trigger' [524416] [1]]

In [34]:
bsp.textures[1].flags = bsp.textures[3].flags
bsp.textures[1].contents = bsp.textures[3].contents

In [39]:
bsp.textures[1].flags = 0x2
bsp.textures[1].contents = 1

In [ ]:
list([(bsp.planes[x.plane].normal, bsp.planes[x.plane].dist) for x in bsp.nodes])

In [ ]:
list(enumerate(bsp.leafs))

In [ ]:
[(i, bsp.faces[bsp.leaffaces[x]]) for i, l in enumerate(bsp.leafs) for x in range(l.leafface, l.leafface + l.n_leafface)]

In [8]:
bsp.models

[model: (0, 0, -1033895936) (1141899264, 1141899264, 1131937792) f0  [14] b0  [10]]

In [81]:
cl = {x.cluster: i for i, x in enumerate(bsp.leafs)}
[(i, [cl[y] for y in bsp.visdata.visible(x.cluster)]) for i, x in enumerate(bsp.leafs) if x.cluster >= 0]

[(0, [4, 11]), (4, [4, 11]), (7, [7, 11]), (11, [4, 7, 11])]

In [85]:
bsp.planes[0] = bsp.planes[1]

In [40]:
with open("/home/rantrave/games/Quake3/baseq3/maps/detailtest2.bsp", "wb") as f:
    f.write(bsp.to_bytes())

0x49425350 46 
144, 93  
237, 288  
525, 864  
1389, 468  
1857, 720  
2577, 160  
2737, 112  
2849, 40  
2889, 132  
3021, 560  
3581, 3784  
7365, 180  
7545, 0  
7545, 1976  
9521, 98304  
107825, 1344  
109169, 32 


In [1]:
import bpy
import bmesh
import mathutils
eps = 1e-5

In [111]:
def create_test_bmesh(size):
    m = bmesh.new()
    # matrix=mathutils.Matrix.Translation((-size/2, -size/2, -size/2))
    bmesh.ops.create_cube(m, size=size, matrix=mathutils.Matrix.Scale(0.5, 4, (0, 0, 1)), calc_uvs=True)
    for f in m.faces:
        f.normal_flip()
    bmesh.ops.create_cube(m, size=size/2, calc_uvs=True)
    return m
def get_plane(face):
    return (face.normal, face.normal @ face.verts[0].co)

def select_faces(bm, pred, faces=None):
    tmp = bm.copy()
    try:
        bmesh.ops.delete(tmp, geom=[x for x in tmp.faces if (faces is not None and x not in faces) or not pred(x)], context='FACES')
        return tmp
    except:
        tmp.free()
        del tmp
        raise

def get_faces_geom(faces):
    return [y  for x in faces for y in x.verts[:] + x.edges[:] + [x]]

def cut_face(bm, face, no, co, clean_outer=True):
    bmesh.ops.remove_doubles(bm, verts=face.verts[:], dist=1e-3)
    g = bmesh.ops.bisect_plane(bm, geom=[face] + face.edges[:] + face.verts[:], plane_co=co, plane_no=no, clear_inner=not clean_outer, clear_outer=clean_outer)
    nfs = list([x for x in g['geom'] if x.__class__.__name__ == 'BMFace' and x.calc_area() >= eps])
    # remove empty faces
    bmesh.ops.delete(bm, geom=[x for x in g['geom'] if x.__class__.__name__ == 'BMFace' and x.calc_area() < eps], context='FACES')
    if len(nfs)>0:
        return nfs[0]
    else:
        return face if face.is_valid else None

def copy_face(face, dst_mesh):
    return dst_mesh.faces.new([dst_mesh.verts.new(x.co) for x in face.verts])

def intersect_coplanar_convex_faces(bm, f0, f1, eps=1e-5):
    nf = f1.copy()
    for e in f0.edges:
        n = f0.normal.cross(e.verts[1].co - e.verts[0].co).normalized()
        nf = cut_face(bm, nf, n, e.verts[0].co, False)
        # probably move to the end
        # bmesh.ops.remove_doubles(bm, verts=nf.verts[:], dist=1e-3)
        # g = bmesh.ops.bisect_plane(bm, geom=[nf] + nf.edges[:] + nf.verts[:], plane_co=e.verts[0].co, plane_no=n, clear_inner=True, clear_outer=False)
        # nfs = list([x for x in g['geom'] if x.__class__.__name__ == 'BMFace' and x.calc_area() >= eps])
        # # remove empty faces
        # bmesh.ops.delete(bm, geom=[x for x in g['geom'] if x.__class__.__name__ == 'BMFace' and x.calc_area() < eps], context='FACES')
        # # print(nfs)
        # if len(nfs)>0:
        #     nf = nfs[0]
        # else:
        #     nf = nf if nf.is_valid else None
        if nf is None:
            # print("nf is none")
            return None
    area = nf.calc_area()
    if abs(area) < eps:
        return None
    if abs(f0.calc_area() - area) < eps:
        bmesh.ops.delete(bm, geom=[nf], context='FACES')
        return f0
    if abs(f1.calc_area() - area) < eps:
        bmesh.ops.delete(bm, geom=[nf], context='FACES')
        return f1
    return nf

def get_planar_intersections(bm):
    bmesh.ops.triangulate(bm, faces=bm.faces[:])
    bm.faces.ensure_lookup_table()
    bm.faces.index_update()
    for i in bm.faces:
        i.normal_update()
    faces = bm.faces[:]
    tf = [list([(x, i) for i, x in enumerate(faces)]), []]
    inters = set()
    used = set()
    for i in range(len(faces)):
        for j in range(i+1, len(faces)):
            f = intersect_coplanar_convex_faces(bm, faces[i], faces[j])
            if f is not None and f != faces[i] and f != faces[j]:
                used.add((i, j))
                tf[1].append((f, (i, j)))
                inters.add(i)
                inters.add(j)
    p = 1 
    while len(tf[p]) > 0:
        newfaces = []
        for face, inds in tf[p]:
            face.normal_update()
            for i in inters:
                if i in inds: continue
                q = tuple(sorted(inds + [i]))
                if q in used: continue
                f = intersect_coplanar_convex_faces(bm, face, faces[i])
                if f is not None:
                    newfaces.append((f, q))
        tf.append(newfaces)
        p += 1
    print(tf)
    return [[y for y, *_ in x] for x in tf]

def get_planar_area(bm):
    sgn = 1
    sum = 0
    for i in get_planar_intersections(bm):
        for j in i:
            sum += sgn * j.calc_area()
        sgn *= -1
    return sum

def is_coplanar(face, plane):
    return abs(face.normal @ plane[0]) < eps * eps and classify(plane, face.verts[0].co) == 0
    
def check_portal(bm, portal_face, portal_plane, faces):
    tmp_bm = select_faces(bm, lambda f: is_coplanar(f, portal_plane), faces)
    try:
        for j in portal_face.edges:
            n = portal_face.normal.cross(j.verts[1].co - j.verts[0].co).normalized()
            bmesh.ops.bisect_plane(tmp_bm, geom=tmp_bm.faces[:] + tmp_bm.edges[:] + tmp_bm.verts[:], plane_co=j.verts[0].co, plane_no=n, clear_inner=True)

        area = get_planar_area(tmp_bm)
        p_area = portal_face.calc_area()
        return abs(area - p_area) < eps
    finally:
        tmp_bm.free()
        del tmp_bm

def cut_volume(bm, plane, clean_outer=True):
    bmesh.ops.bisect_plane(bm, geom=bm.faces[:] + bm.edges[:] + bm.verts[:], plane_co=plane[0] * plane[1], plane_no=plane[0], clear_inner=not clean_outer, clean_outer=clean_outer)
    bmesh.ops.holes_fill(bm, edges=bm.edges, sides=len(bm.edges))
    bmesh.ops.delete(bm, geom=[x for x in bm.faces if x.calc_area() < eps], context='FACES')
    for face in bm.faces:
        bmesh.ops.remove_doubles(bm, verts=face.verts[:], dist=eps)


def get_extents(bm, size):
    xmin = size
    ymin = size
    zmin = size
    xmax = -size
    ymax = -size
    zmax = -size
    for v in bm.verts:
        xmin = min(xmin, v.co.x)
        ymin = min(ymin, v.co.y)
        zmin = min(zmin, v.co.z)
        xmax = max(xmax, v.co.x)
        ymax = max(ymax, v.co.y)
        zmax = max(zmax, v.co.z)
    return (xmin, ymin, zmin), (xmax, ymax, zmax)

def compare_planes(x, y):
    return all([abs(xx - yy) < eps for xx,yy in zip(x[0],y[0])]) and abs(x[1] - y[1]) < eps

def build_brush_planes_and_extents(bm, size):
    ext = get_extents(bm, size)
    planes = []
    # extents
    planes.append((mathutils.Vector((-1., 0., 0.)), -ext[0][0]))
    planes.append((mathutils.Vector(( 1., 0., 0.)),  ext[1][0]))
    planes.append((mathutils.Vector((0., -1., 0.)), -ext[0][1]))
    planes.append((mathutils.Vector((0.,  1., 0.)),  ext[1][1]))
    planes.append((mathutils.Vector((0., 0., -1.)), -ext[0][2]))
    planes.append((mathutils.Vector((0., 0.,  1.)),  ext[1][2]))

    # faces
    for f in bm.faces:
        planes.append(get_plane(f))

    # projections
    tmp = bmesh.new()
    try:
        ones = mathutils.Vector((1., 1., 1.))
        axes = [
            mathutils.Vector((1., 0., 0.)),
            mathutils.Vector((0., 1., 0.)),
            mathutils.Vector((0., 0., 1.)),
        ]
        for a in axes:
            tmp.clear()
            ort = ones - a

            for v in bm.verts:
                tmp.verts.new(v.co * ort)
            bmesh.ops.remove_doubles(tmp, verts=tmp.verts[:], dist=eps)
            bmesh.ops.convex_hull(tmp, input=tmp.verts[:])
            f = bmesh.ops.join_triangles(tmp, faces=tmp.faces, cmp_seam=False,cmp_sharp=False,cmp_uvs=False,cmp_vcols=False,angle_face_threshold=10,angle_shape_threshold=10)
            if len(f["faces"]) == 0:
                # face is segment or point
                if len(tmp.verts) < 2:
                    # projected to point, ignore
                    continue
                tmp.verts.ensure_lookup_table()
                n = a.cross(tmp.verts[0].co - tmp.verts[1].co).normalized()
                d = n @ tmp.verts[0].co
                planes.append((n, d))
                n = -n
                d = n @ tmp.verts[0].co
                planes.append((n, d))
                continue
            f = f['faces'][0]

            for e in f.edges:
                n = -f.normal.cross(e.verts[1].co - e.verts[0].co).normalized()
                d = n @ e.verts[0].co
                planes.append((n, d))
    finally:
        tmp.free()
        del tmp

    return [v1 for i, v1 in enumerate(planes) if not any((compare_planes(v1, v2) for v2 in planes[:i]))]

def classify(plane, co):
    d = plane[0] @ co - plane[1]
    if d > eps: return 1
    if d < -eps: return 2
    return 0

def classify_face(plane, face):
    dst = 0
    for v in face.verts:
        dst = dst | classify(plane, v.co)
        if dst == 3: break
    return dst

def check_intersection(brush, faces):
    for p in brush:
        for f in face:
            if classify_face(p, f) == 1: return True
    return False
            


In [89]:
from typing import Tuple, List, Iterable

def cat(*params: Iterable):
    for i in params:
        for j in i:
            yield j

def create_plane(bm, no, d, size):
    n = mathutils.Vector(no)
    inv = 1.0 / (1 + n.z)
    dd = - n.x * n.y * inv
    v0 = size * mathutils.Vector((1 - n.x * n.x * inv, dd, n.x))
    v1 = size * mathutils.Vector((dd, 1 - n.y * n.y * inv, n.y))
    f = bm.faces.new((
        bm.verts.new(-v0-v1 + d * n),
        bm.verts.new( v0-v1 + d * n),
        bm.verts.new( v0+v1 + d * n),
        bm.verts.new( v1-v0 + d * n)))
    f.normal_update()
    return f

class bsp_tree:
    def __init__(self, mesh):
        self.mesh = mesh
        self.mesh.faces.ensure_lookup_table()
        self.portals_mesh = bmesh.new()
        self.size = 0
        for i in self.mesh.verts:
            self.size = max(i.co.length, self.size)
        self.root = None

    def build(self, hints=[]):
        self.root = bsp_node(self, self.mesh.faces)
        self.root.split(hints)
        self.root.leafize()

        self.__numerate()
        self.__build_portals()
        self.__mark_areas()
        self.__cut_volumes()

    def add_detail(self, faces):
        pass
    def add_brushes(self, brushes):
        pass
    def get_node(self, index):
        if index < 0:
            return self.leafs[index]
        return self.nodes[index]

    def _pick_splitter(self, faces):
        return faces[0]
        # for i in faces:
            # if i in used_faces:
                # continue
            # return i

    def _add_portal_plane(self, plane):
        return create_plane(self.portals_mesh, plane[0], plane[1], self.size)

    def _cut_portal(self, face, plane, front=True):
        return cut_face(self.portals_mesh, face, plane[0], plane[1], not front)

    def __build_portals(self):
        leafs = self.root.list_leafs()
        for i in range(len(leafs)):
            leafs[i].cut_volume()
            for j in range(i+1, len(leafs)):
                prt, node = leafs[i].find_portal(leafs[j])
                is_closed = check_portal(self.portals_mesh, prt, node.p, leafs[i].faces)
                if prt is None: continue
                leafs[i].add_portal(prt, leafs[j], is_closed)
                leafs[j].add_portal(prt, leafs[i], is_closed)

    def __numerate(self):
        self.nodes = [self.root]
        self.leafs = []
        q = [self.root]
        self.root.set_index(0)
        node_index = 1
        leaf_index = 0
        while len(q) > 0:
            n: bsp_node = q.pop()
            if n.is_leaf():
                n.set_index(-1-leaf_index)
                leaf_index += 1
                self.leafs.append(n)
                continue
            n.set_index(node_index)
            node_index += 1
            self.nodes.append()
            for i in n.children:
                q.append(i)


    def __mark_areas(self):
        unused = set(range(len(self.leafs)))
        p = mathutils.Vector((-self.size * 2, -self.size * 2, -self.size * 2))
        # mark external (definetly -1)
        q = [self.root.find(p)]
        while len(q) > 0:
            n = q.pop()
            unused.remove(n.index)
            n.area = -1
            for _, neighbour in n.open_portals():
                if neighbour in unused:
                    q.append(neighbour)

        # rest is probably inside
        area = 0
        while len(unused) > 0:
            q = [unused.pop()]
            while len(q) > 0:
                n = q.pop()
                unused.remove(n.index)
                n.area = area
                for _, neighbour in n.open_portals():
                    if neighbour in unused:
                        q.append(neighbour)
            area += 1
            
    def __cut_volumes(self):
        for i in self.leafs:
            i.cut_volume()

class penumbra:
    def __init__(self, node, portal0, node0, portal1, node1):
        self._mesh = bmesh.new()

        self.node = node
        
        self.p0 = copy_face(portal0, self._mesh)
        self.p1 = portal1

class bsp_node:
    def __init__(self, tree: bsp_tree, faces, parent=None, depth=0):
        self.tree = tree
        self.faces = faces[:]
        self.p = None
        self.children = None
        self.parent = parent
        self.depth = depth
        self.portals_open = []
        self.portals_closed = []
        self.area = 0
        self.volume = None
        self.index = None
        self.detail_faces = []
        self.brushes = []

    def set_index(self, index):
        self.index = index

    def get_index(self):
        return self.index

    def is_front(self):
        if self.parent is None: return True
        return self.parent.children[0] == self

    def is_leaf(self):
        return self.children is None

    def add_portal(self, face, other_leaf, closed):
        if closed:
            self.portals_closed.append((face, other_leaf))
        else:
            self.portals_open.append((face, other_leaf))

    def open_portals(self):
        for i in self.portals_open:
            yield i

    def split(self, hints=[]):
        # check if leaf
        if len(self.faces) == 0: return
        if len(hints) > 0:
            s = None
            p = hints[0]
        else:
            s = self.tree._pick_splitter(self.faces)
            p = get_plane(s)
        self.portal_face = self.tree._add_portal_plane(p)
        pr = self.parent
        while pr is not None:
            self.portal_face = self.tree._cut_portal(self.portal_face, pr.p, self.is_front())
            pr = pr.parent
        self.p = p
        front = []
        back = []
        own = [] if s is None else [s]
        for f in self.faces:
            if f == s: continue
            d = classify_face(p, f)
            if d & 0x1: front.append(f)
            if d & 0x2: back.append(f)
            if d == 0: own.append(f)
        
        self.faces = own
        print(self.faces)
        self.children = [bsp_node(self.tree, front, self, self.depth + 1), bsp_node(self.tree, back, self, self.depth + 1)]

        for c in self.children:
            c.split(hints[1:])

    def find(self, co):
        if self.is_leaf():
            return self

        c = classify(self.p, co)
        if c & 0x1:
            return self.children[0].find(co)
        
        return self.children[1].find(co)

    def leafize(self, faces = []):
        if self.is_leaf():
            self.faces += faces
            return
        front = []
        back = []
        for i in faces:
            d = self.tree.classify(self.p, i)
            print(i, d)
            if (d != 0x2): front.append(i) # incident, front or coplanar
            if (d != 0x1): back.append(i) # incident, back or coplanar

        self.children[0].leafize(front + self.faces)
        self.children[1].leafize(back + self.faces)
        self.faces = []

    # def build_penumbras(self):

    def cut_volume(self):
        if not self.is_leaf(): return
        self.volume = bmesh.new()
        bmesh.ops.create_cube(self.volume, size=self.tree.size * 2)
        n = self
        while n.parent is not None:
            cut_volume(self.volume, self.parent.p, not self.is_front())
            n = n.parent

    def lca(self, node):
        npath = []
        spath = []
        na = node
        sa = self
        while(na != sa):
            if na.depth > sa.depth:
                na = na.parent
                npath.append(na)
                continue
            if na.depth < sa.depth:
                sa = sa.parent
                spath.append(sa)
                continue
            na = na.parent
            sa = sa.parent
            npath.append(na)
            spath.append(sa)
        return sa, (spath, npath)

    # def list_leafs(self):
    #     leafs = []
    #     q = [self]
    #     while len(q) > 0:
    #         s = q.pop()
    #         if s.children is None:
    #             leafs.append(s)
    #             continue
    #         q += s.children

    #     return leafs

    def find_portal(self, to_leaf):
        ca, (n0, n1) = self.lca(to_leaf)
        if ca.portal_face is None:
            return None
        portal = ca.portal_face
        if portal is None:
            raise LookupError("Invalid portal")
        portal = portal.copy()
        for i in cat(n0, n1):
            portal = tree._cut_portal(portal, i.p, i.is_front())
            if portal is None:
                return None

        return portal, ca

    def str(self, tabs=0):
        if self.children is None:
            return f"{' ' * tabs}Leaf: {self.faces}"
        return f"{' '* tabs}- {self.p}\n{' '* tabs}{self.children[0].str(tabs+2)}\n{' '* tabs}{self.children[1].str(tabs+2)}"
        


In [141]:
tree = bsp_tree(m)

In [142]:
root = bsp_node(tree, list(range(0, len(m.faces))))

In [3]:
q = []
q[1:]

[]

In [144]:
root.leafize()

(Vector((1.0, 0.0, 0.0)), -256.0) []
-------------------------
-------------------------
[]
[]
[0]
-------------------------
(Vector((0.0, -1.0, -0.0)), -256.0) [0]
-------------------------
0 1
-------------------------
[0]
[]
[1]
-------------------------
(Vector((-1.0, 0.0, -0.0)), -256.0) [0, 1]
-------------------------
0 1
1 1
-------------------------
[0, 1]
[]
[2]
-------------------------
(Vector((0.0, 1.0, 0.0)), -256.0) [0, 1, 2]
-------------------------
0 1
1 1
2 1
-------------------------
[0, 1, 2]
[]
[3]
-------------------------
(Vector((0.0, 0.0, 1.0)), -128.0) [0, 1, 2, 3]
-------------------------
0 1
1 1
2 1
3 1
-------------------------
[0, 1, 2, 3]
[]
[4, 10]
-------------------------
(Vector((0.0, 0.0, -1.0)), -128.0) [0, 1, 2, 3, 4, 10]
-------------------------
0 1
1 1
2 1
3 1
4 1
10 1
-------------------------
[0, 1, 2, 3, 4, 10]
[]
[5, 11]
-------------------------
(Vector((-1.0, -0.0, 0.0)), 128.0) [0, 1, 2, 3, 4, 10, 5, 11]
-------------------------
0 1
1 

In [25]:
import time
import random as r

test = 10000
b = bmesh.new()
bmesh.ops.create_cone(b, cap_ends=True, segments=256, diameter1=64, diameter2=0, depth=128)
t = time.time()
n = mathutils.Vector((0, 0, 1))
tt0 = 0.
tt1 = 0.
tt2 = 0.

for i in range(test):
    t0 = time.time()
    p0 = mathutils.Vector((0, 0, 0.1 + 0.85 * (test - i) / test))
    t1 = time.time()
    q = b.faces[:] + b.edges[:] + b.verts[:]
    t2 = time.time()
    bmesh.ops.bisect_plane(b, geom=q, plane_co=p0, plane_no=n, clear_outer=True)
    t3 = time.time()
    tt0 += t1 - t0
    tt1 += t2 - t1
    tt2 += t3 - t2

    # bmesh.ops.holes_fill(b, edges=b.edges, sides=1000)

print (f"Total: {tt0}, {tt1}, {tt2}")

Total: 0.0072023868560791016, 0.5723669528961182, 3.649940013885498


In [137]:
b = bmesh.new()
f0 = create_plane(b, (1, 0, 0), 2, 2)
f1 = create_plane(b, (-1, 0, 0), 2, 2)
f2 = create_plane(b, (0, 1, 0), 2, 2)
f3 = create_plane(b, (0, -1, 0), 2, 2)

In [138]:
fc = cut_plane(b, f0, mathutils.Vector((1, 0, 0)), 2, False)
# cut_plane(b, f1, mathutils.Vector((0, 0, 1)), 0, False)

{'geom_cut': [], 'geom': [<BMVert(0x56374c500a20), index=0>, <BMVert(0x56374c500a58), index=1>, <BMVert(0x56374c500a90), index=2>, <BMVert(0x56374c500ac8), index=3>, <BMEdge(0x56374c551e50), index=0, verts=(0x56374c500ac8/3, 0x56374c500a20/0)>, <BMEdge(0x56374c551ea0), index=1, verts=(0x56374c500a20/0, 0x56374c500a58/1)>, <BMEdge(0x56374c551ef0), index=2, verts=(0x56374c500a58/1, 0x56374c500a90/2)>, <BMEdge(0x56374c551f40), index=3, verts=(0x56374c500a90/2, 0x56374c500ac8/3)>, <BMFace(0x56374c4e4cc0), index=0, totverts=4>]}


In [130]:
f0.copy()

<BMFace(0x56374c518bf8), index=-1, totverts=4>

In [139]:
[[y.co for y in x.verts] for x in b.faces[:]]

[[Vector((2.0, -2.0, -2.0)),
  Vector((2.0, -2.0, 2.0)),
  Vector((2.0, 2.0, 2.0)),
  Vector((2.0, 2.0, -2.0))],
 [Vector((-2.0, -2.0, 2.0)),
  Vector((-2.0, -2.0, -2.0)),
  Vector((-2.0, 2.0, -2.0)),
  Vector((-2.0, 2.0, 2.0))],
 [Vector((-2.0, 2.0, -2.0)),
  Vector((2.0, 2.0, -2.0)),
  Vector((2.0, 2.0, 2.0)),
  Vector((-2.0, 2.0, 2.0))],
 [Vector((-2.0, -2.0, 2.0)),
  Vector((2.0, -2.0, 2.0)),
  Vector((2.0, -2.0, -2.0)),
  Vector((-2.0, -2.0, -2.0))]]

In [124]:
[y.co for y in fc.verts]

[Vector((2.0, -2.0, 1.0)),
 Vector((2.0, -2.0, 2.0)),
 Vector((2.0, 2.0, 2.0)),
 Vector((2.0, 2.0, 1.0))]

In [93]:
f0.__class__.__name__ == 'BMFace'

True

In [113]:
m = create_test_bmesh(256)

In [127]:
layer = m.loops.layers.uv.verify()
layer

In [125]:
m.calc_loop_triangles()

[(<BMLoop(0x55fb715e1080), index=-1, vert=0x55fb710bdac0/0, edge=0x55fb710dbcf0/0, face=0x55fb710fbcf0/0>,
  <BMLoop(0x55fb715e1158), index=-1, vert=0x55fb710bdb30/2, edge=0x55fb710dbde0/3, face=0x55fb710fbcf0/0>,
  <BMLoop(0x55fb715e1110), index=-1, vert=0x55fb710bdb68/3, edge=0x55fb710dbd90/2, face=0x55fb710fbcf0/0>),
 (<BMLoop(0x55fb715e1080), index=-1, vert=0x55fb710bdac0/0, edge=0x55fb710dbcf0/0, face=0x55fb710fbcf0/0>,
  <BMLoop(0x55fb715e1110), index=-1, vert=0x55fb710bdb68/3, edge=0x55fb710dbd90/2, face=0x55fb710fbcf0/0>,
  <BMLoop(0x55fb715e10c8), index=-1, vert=0x55fb710bdaf8/1, edge=0x55fb710dbd40/1, face=0x55fb710fbcf0/0>),
 (<BMLoop(0x55fb715e11a0), index=-1, vert=0x55fb710bdb30/2, edge=0x55fb710dbe30/4, face=0x55fb710fbd28/1>,
  <BMLoop(0x55fb715e1278), index=-1, vert=0x55fb710bdc10/6, edge=0x55fb710dbed0/6, face=0x55fb710fbd28/1>,
  <BMLoop(0x55fb715e1230), index=-1, vert=0x55fb710bdc48/7, edge=0x55fb710dbe80/5, face=0x55fb710fbd28/1>),
 (<BMLoop(0x55fb715e11a0), index=-

In [131]:
for f in m.faces:
    print(f"{f}")
    for l in f.loops:
        uv = l[layer].uv
        print(dir(l))
        print(f"{l.vert} {l.vert.index}")
        print(f"{uv}")

<BMFace(0x55fb710fbcf0), index=0, totverts=4>
['__class__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'calc_angle', 'calc_normal', 'calc_tangent', 'copy_from', 'copy_from_face_interp', 'edge', 'face', 'index', 'is_convex', 'is_valid', 'link_loop_next', 'link_loop_prev', 'link_loop_radial_next', 'link_loop_radial_prev', 'link_loops', 'tag', 'vert']
<BMVert(0x55fb710bdac0), index=0> 0
<Vector (0.0000, 0.0000)>
['__class__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__set